# Sonde3 
##  Reads and converts binary water quality environmental instrument data to a DataFrame


### I.  Example Usage

Lets dive in!  

We have a example water quality instrument binary file `"tests/ysi_test_files/SA08.dat"`.  This file was generated by a YSI 600LS instrument and is in proprietary binary format.

#### Using the `sonde()` function we:

1.  `autodetect()` the file type and pass to the correct parser function 
2.  `read_ysi()` the binary file and convert to pandas DataFrame
3.  Transform all datetimes to the UTC timezone
4.  Standardize the units to metric and rename the columns to standard name conventions
3.  Pass the DataFrame to `calculate_salinity_psu()` and `calculate_do_mgl()` to apply standard formulas to generate the salinity and dissolved oxygen columns.

In [1]:
import sonde3
import pandas
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", remove_invalids=False, twdbparams=True)

/home/eturner/sonde3/sonde3/sonde.py:19: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi(filename, tzinfo)
/home/eturner/.local/lib/python3.6/site-packages/seawater/library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


#### Why the runtime warnings?

1.  The YSI instrument files don't contain any timezone information.  Therefore, the function has to assume that the timezone of the file to make the UTC conversion.

2. Often raw instrument files will contain impossible & incorrect values in the beginning and end of the file.  Examples: negative values for salinity or dissolved oxygen percentage.  `sonde3` does not trim the raw file, or perform QA analysis.  `sonde3` will pass the values as they were recorded by the instrument.

To automatically convert invalid negative values to zero, either set the flag **remove_invalids=False** or remove it altogether, as this is the default behavior of the package

##### We can now interact with the two dataframes produced by `sonde3`:


In [2]:
df.info() 
df.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
Datetime_(UTC)                               700 non-null datetime64[ns, UTC]
water_temperature                            700 non-null float64
water_electrical_conductivity                700 non-null float64
water_depth_nonvented                        700 non-null float64
water_dissolved_oxygen_percent_saturation    700 non-null float64
instrument_battery_voltage                   700 non-null float64
water_specific_conductance                   700 non-null float64
seawater_salinity                            678 non-null float64
water_dissolved_oxygen_concentration         678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(8)
memory usage: 49.3 KB


,Datetime_(UTC),water_temperature,water_electrical_conductivity,water_depth_nonvented,water_dissolved_oxygen_percent_saturation,instrument_battery_voltage,water_specific_conductance,seawater_salinity,water_dissolved_oxygen_concentration
0,2008-07-16 12:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 13:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 14:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 15:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 16:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


In [3]:
metadata

,Model,Manufacturer,Instrument_Serial_Number,Station,Deployment_Setup_Time,Filename,Deployment_Start_Time,Deployment_Stop_Time
0,600,YSI,1012,SANT_CDT,,SA08.dat,2008-07-16 12:00:31+00:00,2008-08-14 15:00:16+00:00


### II.  Working with time zones


What if data was collected outside of US/Central time?  Pass the timezone information to `sonde3.sonde`:

In [4]:
import pytz
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", pytz.timezone('US/Eastern'))
df.head()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU,water_DO_mgl
0,2008-07-16 11:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 12:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 13:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 14:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 15:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


### III. Autodetecting files



Curious about what kind of instrument files you have in a directory?  Apply the `sonde3.autodetect` method:

```python
#what kind of file is this??
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 
```

In [5]:
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 

'greenspan_xls'

Copy this code snippet to the notebook line below to run `autodetect()` on all of the use-case examples in the `sonde3` package:

```python
#this script runs through all of the text examples and prints out the autodetect results
import os

root_dir = 'tests'
results = []
for directory, subdirectories, files in os.walk(root_dir):
    for file in files:
        if "_test.txt" in file:
            continue
        os.path.join(directory, file)
        results.append(os.path.join(directory, file) + ' ' + sonde3.autodetect(os.path.join(directory, file)))

results 
```

### IV. Generating Salinity and Dissolved Oxygen

Typically deployed water quality instruments do not compute all rows of data internally.  Instead, these are calculated by the program used to read the file back at the lab.  For example, YSI instruments do not compute salinity or dissolved oxygen concentration.  This can cause confusion because when viewing a YSI *\*.dat* file in YSI's ECOWIN, or Ecowatch Lite program displays salinity and DO mg/L.  However, the raw binary file does not include these rows as they were not physically collected by the instrument during deployment.

For example, lets read the raw binary file of the example file `"tests/ysi_test_files/SA08.dat"` and see what it contains:

In [6]:
metadata, SA08_BIN = sonde3.read_ysi("tests/ysi_test_files/SA08.dat",pytz.timezone('US/Central'))
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 6 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 32.9 KB


The pandas DataFrame contains the columns for conductivity and % DO saturation, but not for salinity and DO concentration.

For comparision, lets read the comma separated version of this file that was produced by the proprietary YSI Ecowin program:

In [7]:
metadata, SA08_CSV = sonde3.read_ysi_ascii("tests/ysi_test_files/SA08.CDF", pytz.timezone('US/Central'),delim=",")


The ECOWIN exported comma separated file has far more columns! The extra columns were derived through formulas using the instrumnet observed measurments.

To generate these columns pass the SA08_BIN DataFrame to calculate Salinity (PSU) and Dissolved Oxygen (mg/L) using `sonde3.calculate_salinity_psu()` and `sonde3.calculate_do_mgl()`

We can then compare our computed results to the ECOwatch program results:

In [8]:
SA08_BIN = sonde3.calculate_salinity_psu(SA08_BIN)
SA08_BIN = sonde3.calculate_do_mgl(SA08_BIN)
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
water_salinity_PSU            678 non-null float64
water_DO_mgl                  678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 43.8 KB


So, why does `sonde3` produce less values for salinity and DO than the comma separated file?  Is this a bug?

*NO!*  If you recal from the `sqrt()` warning above, some of the values in the file are invalid.  This produces null (**NaN**) values in those rows.

In [9]:
SA08_BIN.tail()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
695,2008-08-14 11:00:16+00:00,23.977051,-0.000006,-0.052356,-0.006866,5.546875,NaN,NaN
696,2008-08-14 12:00:16+00:00,23.938599,-0.000005,-0.050262,-0.006866,5.468750,NaN,NaN
697,2008-08-14 13:00:16+00:00,24.039307,-0.000005,-0.045875,-0.006866,5.468750,NaN,NaN
698,2008-08-14 14:00:16+00:00,24.107056,-0.000005,-0.038671,-0.006866,5.468750,NaN,NaN
699,2008-08-14 15:00:16+00:00,24.177551,-0.000006,-0.034785,-0.006866,5.468750,NaN,NaN


Lets check our two files to see if our conversion methods are those used by YSI.  

Lets use `numpy.random` to pick a row to check:



In [10]:
from numpy import random
row = random.randint(2,677) # pick random row

sub1 =  SA08_BIN.iloc[row:row+1]  #row one binary file
sub2 = SA08_CSV.iloc[row:row+1]   #row two csv file

pandas.concat([sub1, sub2], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
293,2008-07-28 17:00:31+00:00,29.390564,35.219727,0.558945,80.425262,5.9375,20.184923,5.497003
293,2008-07-28 17:00:31+00:00,29.390000,35.220000,0.559000,80.400000,5.9000,20.190000,5.500000


Looks the same!  Notice, however, that the csv file values are rounded due to the fixed digit precision.

### V. Files from other Manufacturers and Models
#### a. Hydrotech

In [11]:
metadata, mydf = sonde3.sonde("tests/hydrotech_test_files/0109DELT.CSV", pytz.timezone('US/Central'))

In [12]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Hydrotech,081107-D,MiniSonde4a,0109delt,2009-01-13 06:00:00+00:00,2009-01-14 12:51:00+00:00,2009-02-12 15:51:00+00:00,NaN


In [13]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 13 to 712
Data columns (total 7 columns):
Datetime_(UTC)                       700 non-null datetime64[ns, UTC]
water_temp_C                         700 non-null float64
water_specific_conductivity_mS/cm    700 non-null float64
water_depth_m_nonvented              700 non-null float64
instrument_battery_voltage           700 non-null float64
water_conductivity_mS/cm             700 non-null float64
water_salinity_PSU                   700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 43.8 KB


In [14]:
mydf.head()

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_salinity_PSU
13,2009-01-14 12:51:00+00:00,12.81,0.866,-0.0,6.0,0.664370,0.428305
14,2009-01-14 13:51:00+00:00,12.57,0.868,-0.0,6.0,0.661926,0.429304
15,2009-01-14 14:51:00+00:00,12.34,0.867,-0.0,6.0,0.657354,0.428759
16,2009-01-14 15:51:00+00:00,12.59,0.868,-0.0,6.0,0.662257,0.429306
17,2009-01-14 16:51:00+00:00,11.78,0.883,-0.0,6.0,0.660041,0.436875


In [15]:
mydf.tail()

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_salinity_PSU
708,2009-02-12 11:51:00+00:00,19.24,34.9,1.03,3.9,31.060442,22.008874
709,2009-02-12 12:51:00+00:00,19.22,34.9,1.02,3.9,31.047110,22.008973
710,2009-02-12 13:51:00+00:00,19.31,35.1,1.02,3.9,31.285367,22.147645
711,2009-02-12 14:51:00+00:00,19.44,35.8,1.02,3.9,31.998183,22.634720
712,2009-02-12 15:51:00+00:00,19.34,34.9,1.03,0.0,31.127101,22.008374


#### b. YSI EXO Units

YSI EXO units are timezone aware.  Thus, we discover the internal timezone and convert to UTC if applicable.

In [16]:
sonde3.autodetect("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

'ysi_exo_csv'

In [17]:
#metadata, exo_df = sonde3.sonde("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

In [18]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Hydrotech,081107-D,MiniSonde4a,0109delt,2009-01-13 06:00:00+00:00,2009-01-14 12:51:00+00:00,2009-02-12 15:51:00+00:00,NaN


In [19]:
#exo_df.head()

#### c. YSI ascii + Comma Separated.

YSI timeseries produced by kermit transfer from YSI handset protocol.

In [20]:
metadata, df = sonde3.sonde("tests/ysi_test_files/0917GEB.txt")

/home/eturner/sonde3/sonde3/sonde.py:27: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi_ascii(filename, tzinfo, ',', None, [1, 2, 3])


In [21]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time
0,YSI,,,,,2017-09-08 12:15:08+00:00,2017-09-26 20:00:08+00:00


In [22]:
df.head()

,Datetime_(UTC),water_temp_C,water_tds_g/L,water_pressure_abs,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_specific_conductivity_mS/cm,water_salinity_PSU
0,2017-09-08 12:15:08+00:00,19.93,0.0,14.763,0.303,6.2,0.0,0.0,0.010285
1,2017-09-08 12:30:08+00:00,20.05,0.0,14.767,0.306,6.2,0.0,0.0,0.010336
2,2017-09-08 12:45:08+00:00,20.54,0.0,14.769,0.307,6.2,0.0,0.0,0.010544
3,2017-09-08 13:00:08+00:00,20.91,0.0,14.771,0.309,6.2,0.0,0.0,0.010699
4,2017-09-08 13:15:08+00:00,21.21,0.0,14.774,0.311,6.2,0.0,0.0,0.010823


#### D. Lowell Tiltmeter Comma Separated.

Concatinated timeseries file from a Lowell tiltmeter

In [23]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1603103_SAB_05122016.csv")

/home/eturner/sonde3/sonde3/sonde.py:35: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_lowell(filename, tzinfo, ',')


In [24]:
sonde3.merge_lowell()

In [25]:
metadata.head()

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename
0,Lowell,1603103,TCM,NaN,NaN,2016-05-13 18:21:00+00:00,2016-07-15 18:51:00+00:00,NaN


### VI. Package Validation Tests

#### A.  Check to see if the package is correctly handling daylight savings time.

The test case dataset **tests/ysi_test_files/0108BAYT.csv** was collected during a daylight savings transition.  These three files were processed differently at the time of instrument processing.  The *.dat* file is the raw binary file from the instrument.  The *CDF* file was produced by the Ecowatch program.  The *cvs* was produced by converting the *CDF* file in ms excel to a comma separated file.  If the times were not consistant between the three files it would indicate **sonde3** is not processing datetime correctly.

In [26]:
#import all three versions of the file ,CDF, csv, and binary *.dat
metadata, baytcsv = sonde3.sonde("tests/ysi_test_files/0108BAYT.csv", pytz.timezone('US/Central'))
metadata, baytdat = sonde3.sonde("tests/ysi_test_files/0108BAYT.dat", pytz.timezone('US/Central'))
metadata, baytCDF = sonde3.sonde("tests/ysi_test_files/0108BAYT.CDF", pytz.timezone('US/Central'))

In [27]:
#cut the first & last row from each file.
pandas.concat([baytCDF.iloc[0:1], baytdat.iloc[0:1], baytcsv.iloc[0:1] , \
               baytCDF.iloc[-1:], baytdat.iloc[-1:], baytcsv.iloc[-1:]], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,water_conductivity_mS/cm,water_DO_mgl,water_salinity_PSU
0,2008-01-29 15:00:33+00:00,22.560000,-0.0,-0.0,103.200000,-0.0,8.927547,0.011370
0,2008-01-29 15:00:33+00:00,22.562561,-0.0,-0.0,103.227234,-0.0,8.929467,0.011371
0,2008-01-29 15:00:00+00:00,22.560000,0.0,-0.0,103.200000,0.0,8.927547,0.011370
1061,2008-03-13 19:00:33+00:00,22.510000,-0.0,-0.0,104.300000,-0.0,9.031326,0.011350
1061,2008-03-13 19:00:33+00:00,22.510376,-0.0,-0.0,104.284668,-0.0,9.029934,0.011350
1061,2008-03-13 19:00:00+00:00,22.510000,0.0,-0.0,104.300000,0.0,9.031326,0.011350


#### B.  More parsing testing

In [28]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1708018_3Basin.csv", twdbparams=True)
df.columns

Index(['Datetime_(UTC)', 'water_speed', 'water_bearing',
       'northward_water_velocity', 'eastward_water_velocity',
       'water_temperature'],
      dtype='object')

In [29]:
metadata, df = sonde3.sonde("tests/ysi_test_files/spotchecks-galveston-180618-081639.csv", twdbparams=True)
df.columns

Index(['Datetime_(UTC)', 'water_temperature', 'mmHg',
       'water_dissolved_oxygen_percent_saturation',
       'water_dissolved_oxygen_concentration', 'water_ph', 'water_bga_pe_rfu',
       'chlorophyll_a_rfu', 'water_depth_nonvented',
       'instrument_battery_voltage', 'ALTm', 'Lat', 'Lon',
       'water_specific_conductance', 'water_electrical_conductivity',
       'seawater_salinity'],
      dtype='object')

In [30]:
df

,Datetime_(UTC),water_temperature,mmHg,water_dissolved_oxygen_percent_saturation,water_dissolved_oxygen_concentration,water_ph,water_bga_pe_rfu,chlorophyll_a_rfu,water_depth_nonvented,instrument_battery_voltage,ALTm,Lat,Lon,water_specific_conductance,water_electrical_conductivity,seawater_salinity
0,2018-06-14 09:32:57,29.094,776.4,98.1,6.256748,8.05,2.938,0.523,0.1,2.68,-2.7,29.34172,-94.78282,51.337,55.351317,33.600123
1,2018-06-14 09:33:15,29.002,776.4,97.8,6.245734,8.06,3.334,0.635,0.5,2.70,-4.4,29.34171,-94.78282,51.380,55.307395,33.634646
2,2018-06-14 09:33:27,29.014,776.4,98.0,6.257663,8.07,3.587,0.659,1.0,2.71,-4.5,29.34172,-94.78282,51.366,55.304098,33.623810
3,2018-06-14 09:33:41,28.987,776.4,98.0,6.259997,8.07,3.883,0.711,1.5,2.71,-4.8,29.34171,-94.78281,51.381,55.293750,33.635586
4,2018-06-14 09:33:53,28.965,776.4,97.8,6.249357,8.08,3.868,0.715,2.0,2.71,-3.6,29.34172,-94.78281,51.384,55.275387,33.638362
5,2018-06-14 09:34:16,28.938,776.4,97.6,6.238828,8.09,3.633,0.699,2.5,2.70,-3.3,29.34173,-94.78284,51.402,55.268243,33.652346
6,2018-06-14 09:34:29,28.936,776.4,97.6,6.239084,8.09,3.796,0.727,3.0,2.70,-2.9,29.34172,-94.78284,51.400,55.264129,33.650797
7,2018-06-14 09:34:45,28.933,776.4,97.5,6.232951,8.09,3.599,0.697,4.0,2.71,-3.8,29.34172,-94.78284,51.402,55.263334,33.652086
8,2018-06-14 09:35:01,28.931,776.5,97.3,6.220324,8.11,3.548,0.725,5.0,2.70,-2.9,29.34172,-94.78283,51.404,55.263520,33.653343
9,2018-06-14 09:35:33,28.931,776.4,97.1,6.207872,8.11,4.677,1.031,5.5,2.71,-3.9,29.34173,-94.78284,51.391,55.249544,33.643628


In [31]:
df.columns

Index(['Datetime_(UTC)', 'water_temperature', 'mmHg',
       'water_dissolved_oxygen_percent_saturation',
       'water_dissolved_oxygen_concentration', 'water_ph', 'water_bga_pe_rfu',
       'chlorophyll_a_rfu', 'water_depth_nonvented',
       'instrument_battery_voltage', 'ALTm', 'Lat', 'Lon',
       'water_specific_conductance', 'water_electrical_conductivity',
       'seawater_salinity'],
      dtype='object')